In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np

In [4]:
a = np.array([0,1,1])

In [5]:
a == [0,1,1]

array([ True,  True,  True])

In [8]:
if (a== [0,1,1]).all():
    print('x')

x
